Q. Using a deep learning algorithm of your choice, learn the representation of dark matter in the strong gravitational lensing images provided using PyTorch.
The goal is to use this learned representation for anomaly detection, hence you should pick the most appropriate algorithm and discuss your strategy.   
   
A. Considering the purpose is anomaly detection, I choose Autoencoder as the most appropriate method, as discussed in [1]. I think Conditional VAE (CVAE)[5] especially suits this purpose.   
Deep neural network architecture is known to function relatively better in tasks about images than MLP(multilayer perception), and in the early stage Convolutional AutoEncoder(hereafter CAE)[2] and Variational AutoEncoder(VAE)[8] was the most competent method.
Of cource, it is said that CNN architecture in general performs well, speaking of image processing.  
CAE incorpolates CNN's algorithm into itself, and according to [3],[4] (written in Japnanese), it clearly shows that CAE overwhelms other MLP algorithms.   
VAE incorpolates probability distribution into latent variable z, and it also performs as well as CAE does.
These days, lots of methods derived from these, and I noticed CVAE is the best, which add supervising features to VAE. it is compared with lots of other autoencoder methods on purpose of analysis on astronomical images[6], and prove that the latent variables obtained with CVAE preserve more information and are more discriminative towards real astronomical transients.(SCAE[7] did well as well, but slightly CVAE has an advantage of it)    
This paper was published in April 2018, and it is the most recent paper applying autoencoder to processing astronomical images.(According to my survey on Google Scholor)
In addition to that, CVAE is implementable on laptop, whereas other methods sometimes need a bulk of computer properties(such as [9].)

Using such autoencoders, anomaly detection will be easy.(in future work I can impliment heat map which clearly tells anomalies from norms [10] )

According to Hold-out method, I devided images into train(3001 images) and test(2000 images).  
Directory structures are below. you should put lenses directory at the same place as this notebook. 
- lenses/train/sub/(3001 images)
- lenses/train/no_sub/(3001 images)
- lenses/test/sub/(2000 images)
- lenses/test/no_sub/(2000 images)

About optimizer, I use Adam because this is most popular optimizers' father and since this is not that complicated, the convergence speed is fast.

in inplementation, I refered to this site:  
https://graviraja.github.io/conditionalvae/#


----


Of cource, CNN could be used because this problem can be taken as a classification problem(so no_sub images provided)
~~In case, I also implement CNN, which was written at the end of this notebook.(I used ResNet, as in [11])~~
I couldn't implement CNN case in time.



----

[1]Deep Learning the Morphology of Dark Matter Substructure, pp8  
https://arxiv.org/pdf/1909.07346.pdf

[2]Deep Clustering with Convolutional Autoencoders  
https://link.springer.com/chapter/10.1007/978-3-319-70096-0_39

[3]PytorchによるAutoEncoder Familyの実装(Implementation of various Autoencoders using PyTorch)  
https://elix-tech.github.io/ja/2016/07/17/autoencoder.html

[4]様々なオートエンコーダによる異常検知(Anomaly Detection with lots of Autoencoders)  
https://sinyblog.com/deaplearning/auto_encoder_001/

[5]Semi-Supervised Learning with Deep Generative Models (CVAE)  
https://arxiv.org/abs/1406.5298

[6]Latent representations of transient candidates from an astronomical image difference pipeline using Variational Autoencoders  
https://pdfs.semanticscholar.org/57ce/a9520008706b8cb190d94513e695068210cd.pdf

[7]Convolutional Sparse Autoencoders for Image Classification (SCAE)  
https://ieeexplore.ieee.org/document/7962256

[8]Auto-Encoding Variational Bayes(VAE)  
https://arxiv.org/pdf/1312.6114.pdf

[9]Improving Variational Autoencoder with Deep Feature Consistent and Generative Adversarial Training (GAN)  
https://arxiv.org/pdf/1906.01984.pdf

[10]Anomaly Manufacturing Product Detection using Unregularized Anomaly Score on Deep Generative Models  
https://confit.atlas.jp/guide/event-img/jsai2018/2A1-03/public/pdf?type=in  
https://qiita.com/shinmura0/items/811d01384e20bfd1e035

[11] Deep Learning the Morphology of Dark Matter Substructure https://arxiv.org/abs/1909.07346

# 1: CVAE

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils

import matplotlib.pyplot as plt
import pandas as pd
from skimage import io, transform
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [43]:
BATCH_SIZE = 1         # number of data points in each batch
N_EPOCHS = 1           # times to run the model on complete data
INPUT_DIM = 150 * 150     # size of each input
HIDDEN_DIM = 4        # hidden dimension
LATENT_DIM = 15         # latent vector dimension
N_CLASSES = 2          # number of classes in the data
lr = 1e-3               # learning rate

In [44]:
data_transform = transforms.Compose([transforms.RandomSizedCrop(INPUT_DIM), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder(root='./lenses/train', #3001 images each
                                           transform=data_transform)
test_dataset = datasets.ImageFolder(root='./lenses/test', #2000 images each
                                           transform=data_transform)

In [45]:
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [46]:
def idx2onehot(idx, n=N_CLASSES):

    assert idx.shape[1] == 1
    assert torch.max(idx).item() < n

    onehot = torch.zeros(idx.size(0), n)
    onehot.scatter_(1, idx.data, 1)

    return onehot

In [47]:
class Encoder(nn.Module):
    ''' This the encoder part of VAE
    '''
    def __init__(self, input_dim, hidden_dim, latent_dim, n_classes):
        '''
        Args:
            input_dim: A integer indicating the size of input 
            hidden_dim: A integer indicating the size of hidden dimension.
            latent_dim: A integer indicating the latent size.
            n_classes: A integer indicating the number of classes. (dimension of one-hot representation of labels)
        '''
        super().__init__()

        self.linear = nn.Linear(input_dim + n_classes, hidden_dim)
        self.mu = nn.Linear(hidden_dim, latent_dim)
        self.var = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        # x is of shape [batch_size, input_dim + n_classes]

        hidden = F.relu(self.linear(x))
        # hidden is of shape [batch_size, hidden_dim]

        # latent parameters
        mean = self.mu(hidden)
        # mean is of shape [batch_size, latent_dim]
        log_var = self.var(hidden)
        # log_var is of shape [batch_size, latent_dim]

        return mean, log_var


In [48]:
class Decoder(nn.Module):
    ''' This the decoder part of VAE
    '''
    def __init__(self, latent_dim, hidden_dim, output_dim, n_classes):
        '''
        Args:
            latent_dim: A integer indicating the latent size.
            hidden_dim: A integer indicating the size of hidden dimension.
            output_dim: A integer indicating the size of output 
            n_classes: A integer indicating the number of classes. (dimension of one-hot representation of labels)
        '''
        super().__init__()

        self.latent_to_hidden = nn.Linear(latent_dim + n_classes, hidden_dim)
        self.hidden_to_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x is of shape [batch_size, latent_dim + num_classes]
        x = F.relu(self.latent_to_hidden(x))
        # x is of shape [batch_size, hidden_dim]
        generated_x = F.sigmoid(self.hidden_to_out(x))
        # x is of shape [batch_size, output_dim]

        return generated_x


In [49]:
class CVAE(nn.Module):
    ''' This the VAE, which takes a encoder and decoder.
    '''
    def __init__(self, input_dim, hidden_dim, latent_dim, n_classes):
        '''
        Args:
            input_dim: A integer indicating the size of input.
            hidden_dim: A integer indicating the size of hidden dimension.
            latent_dim: A integer indicating the latent size.
            n_classes: A integer indicating the number of classes. (dimension of one-hot representation of labels)
        '''
        super().__init__()

        self.encoder = Encoder(input_dim, hidden_dim, latent_dim, n_classes)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim, n_classes)

    def forward(self, x, y):

        x = torch.cat((x, y), dim=1)

        # encode
        z_mu, z_var = self.encoder(x)

        # sample from the distribution having latent parameters z_mu, z_var
        # reparameterize
        std = torch.exp(z_var / 2)
        eps = torch.randn_like(std)
        x_sample = eps.mul(std).add_(z_mu)

        z = torch.cat((x_sample, y), dim=1)

        # decode
        generated_x = self.decoder(z)

        return generated_x, z_mu, z_var

In [50]:
def calculate_loss(x, reconstructed_x, mean, log_var):
    # reconstruction loss
    RCL = F.binary_cross_entropy(reconstructed_x, x, size_average=False)
    # kl divergence loss
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

    return RCL + KLD

In [51]:
model = CVAE(INPUT_DIM, HIDDEN_DIM, LATENT_DIM, N_CLASSES)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [52]:
def train():
    # set the train mode
    model.train()

    # loss of the epoch
    train_loss = 0

    for i, (x, y) in enumerate(train_iterator):
        # reshape the data into 
        x = x.view(-1,  INPUT_DIM*3)
        x = x.to(device)

        # convert y into one-hot encoding
        y = idx2onehot(y.view(-1, 1))
        y = y.to(device)

        # update the gradients to zero
        optimizer.zero_grad()

        # forward pass
        reconstructed_x, z_mu, z_var = model(x, y)

        # loss
        loss = calculate_loss(x, reconstructed_x, z_mu, z_var)

        # backward pass
        loss.backward()
        train_loss += loss.item()

        # update the weights
        optimizer.step()

    return train_loss


In [53]:
def test():
    # set the evaluation mode
    model.eval()

    # test loss for the data
    test_loss = 0

    # we don't need to track the gradients, since we are not updating the parameters during evaluation / testing
    with torch.no_grad():
        for i, (x, y) in enumerate(test_iterator):
            # reshape the data
            x = x.view(-1, INPUT_DIM*3) #TASK: modifying shapes of the dataset, data shapes in one batch size must be the same https://nodaki.hatenablog.com/entry/2018/07/18/001851
            x = x.to(device)

            # convert y into one-hot encoding
            y = idx2onehot(y.view(-1, 1))
            y = y.to(device)

            # forward pass
            reconstructed_x, z_mu, z_var = model(x, y)

            # loss
            loss = calculate_loss(x, reconstructed_x, z_mu, z_var)
            test_loss += loss.item()

    return test_loss

In [54]:
best_test_loss = float('inf')

for e in range(N_EPOCHS):

    train_loss = train()
    test_loss = test()

    train_loss /= len(train_dataset)
    test_loss /= len(test_dataset)

    print(f'Epoch {e}, Train Loss: {train_loss:.2f}, Test Loss: {test_loss:.2f}')

    if best_test_loss > test_loss:
        best_test_loss = test_loss
        patience_counter = 1
    else:
        patience_counter += 1

    if patience_counter > 3:
        break

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 22500 and 1 in dimension 0 at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensor.cpp:612

In [ ]:
# create a random latent vector
z = torch.randn(1, LATENT_DIM).to(device)

# pick randomly 1 class, for which we want to generate the data
y = torch.randint(0, N_CLASSES, (1, 1)).to(dtype=torch.long)
print(f'Generating a {y.item()}')

y = idx2onehot(y).to(device, dtype=z.dtype)
z = torch.cat((z, y), dim=1)

reconstructed_img = model.decoder(z)
img = reconstructed_img.view(150, 150).data

plt.figure()
plt.imshow(img, cmap='gray')
plt.show()

#### below are implemented otherwise and nothing to do with CVAE.  


# 2: VAE

In [62]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils

import matplotlib.pyplot as plt
import pandas as pd
from skimage import io, transform
import numpy as np

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [64]:
BATCH_SIZE = 1        # number of data points in each batch
N_EPOCHS = 2           # times to run the model on complete data
INPUT_DIM = 150 * 150     # size of each input
HIDDEN_DIM = 4        # hidden dimension
LATENT_DIM = 15         # latent vector dimension
lr = 1e-3               # learning rate

In [65]:
data_transform = transforms.Compose([transforms.RandomSizedCrop(INPUT_DIM), transforms.ToTensor()])

train_dataset = datasets.ImageFolder(root='./lenses/train', #3001 images each
                                           transform=data_transform)
test_dataset = datasets.ImageFolder(root='./lenses/test', #2000 images each
                                           transform=data_transform)

In [66]:
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [67]:
class Encoder(nn.Module):
    ''' This the encoder part of VAE
    '''
    def __init__(self, input_dim, hidden_dim, z_dim):
        '''
        Args:
            input_dim: A integer indicating the size of input (in case of MNIST 28 * 28).
            hidden_dim: A integer indicating the size of hidden dimension.
            z_dim: A integer indicating the latent dimension.
        '''
        super().__init__()

        self.linear = nn.Linear(input_dim, hidden_dim)
        self.mu = nn.Linear(hidden_dim, z_dim)
        self.var = nn.Linear(hidden_dim, z_dim)

    def forward(self, x):
        # x is of shape [batch_size, input_dim]

        hidden = F.relu(self.linear(x))
        # hidden is of shape [batch_size, hidden_dim]
        z_mu = self.mu(hidden)
        # z_mu is of shape [batch_size, latent_dim]
        z_var = self.var(hidden)
        # z_var is of shape [batch_size, latent_dim]

        return z_mu, z_var

In [68]:
class Decoder(nn.Module):
    ''' This the decoder part of VAE
    '''
    def __init__(self, z_dim, hidden_dim, output_dim):
        '''
        Args:
            z_dim: A integer indicating the latent size.
            hidden_dim: A integer indicating the size of hidden dimension.
            output_dim: A integer indicating the output dimension (in case of MNIST it is 28 * 28)
        '''
        super().__init__()

        self.linear = nn.Linear(z_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x is of shape [batch_size, latent_dim]

        hidden = F.relu(self.linear(x))
        # hidden is of shape [batch_size, hidden_dim]

        predicted = torch.sigmoid(self.out(hidden))
        # predicted is of shape [batch_size, output_dim]
        return predicted


In [69]:
class VAE(nn.Module):
    def __init__(self, enc, dec):
        ''' This the VAE, which takes a encoder and decoder.
        '''
        super().__init__()

        self.enc = enc
        self.dec = dec

    def forward(self, x):
        # encode
        z_mu, z_var = self.enc(x)

        # sample from the distribution having latent parameters z_mu, z_var
        # reparameterize
        std = torch.exp(z_var / 2)
        eps = torch.randn_like(std)
        x_sample = eps.mul(std).add_(z_mu)

        # decode
        predicted = self.dec(x_sample)
        return predicted, z_mu, z_var


In [70]:
# encoder
encoder = Encoder(INPUT_DIM, HIDDEN_DIM, LATENT_DIM)

# decoder
decoder = Decoder(LATENT_DIM, HIDDEN_DIM, INPUT_DIM)

# vae
model = VAE(encoder, decoder).to(device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

In [74]:
def train():
    # set the train mode
    model.train()

    # loss of the epoch
    train_loss = 0

    for i, (x, _) in enumerate(train_iterator):
        # reshape the data into [batch_size, INPUT_DIM]
        x = x.view(-1, INPUT_DIM)
        x = x.to(device)

        # update the gradients to zero
        optimizer.zero_grad()

        # forward pass
        x_sample, z_mu, z_var = model(x)

        # reconstruction loss
        recon_loss = F.binary_cross_entropy(x_sample, x, size_average=False)

        # kl divergence loss
        kl_loss = 0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1.0 - z_var)

        # total loss
        loss = recon_loss + kl_loss

        # backward pass
        loss.backward()
        train_loss += loss.item()

        # update the weights
        optimizer.step()

    return train_loss


In [75]:

def test():
    # set the evaluation mode
    model.eval()

    # test loss for the data
    test_loss = 0

    # we don't need to track the gradients, since we are not updating the parameters during evaluation / testing
    with torch.no_grad():
        for i, (x, _) in enumerate(test_iterator):
            # reshape the data
            x = x.view(-1, INPUT_DIM)
            x = x.to(device)

            # forward pass
            x_sample, z_mu, z_var = model(x)

            # reconstruction loss
            recon_loss = F.binary_cross_entropy(x_sample, x, size_average=False)

            # kl divergence loss
            kl_loss = 0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1.0 - z_var)

            # total loss
            loss = recon_loss + kl_loss
            test_loss += loss.item()

    return test_loss


In [ ]:
best_test_loss = float('inf')

for e in range(N_EPOCHS):

    train_loss = train()
    test_loss = test()

    train_loss /= len(train_dataset)
    test_loss /= len(test_dataset)

    print(f'Epoch {e}, Train Loss: {train_loss:.2f}, Test Loss: {test_loss:.2f}')

    if best_test_loss > test_loss:
        best_test_loss = test_loss
        patience_counter = 1
    else:
        patience_counter += 1

    if patience_counter > 3:
        break

/opt/anaconda3/envs/autolens/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# sample and generate a image
z = torch.randn(1, LATENT_DIM).to(device)
reconstructed_img = model.dec(z)
img = reconstructed_img.view(150, 150).data

print(z.shape)
print(img.shape)
plt.figure()
plt.imshow(img, cmap='gray')
plt.show()




# 3: CNN(incomplete)

### Though it is already implemented in the paper(Deep Learning the Morphology of Dark Matter Substructure), this method is also useful when it comes to learning representation for anomaly detection.

TODO: make labels(or use directory names as labels) according to sub or no_sub

In [ ]:
from cnn_finetune import make_model

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

from PIL import Image
import pandas as pd
import os

In [ ]:
MODEL_NAME = 'pnasnet5large'
CLASSES = ( 'sub', 'no_sub')
BATCH_SIZE = 128
MOMENTUM = 0.9 # for SDG
N_EPOCHS = 10
lr = 1e-3  

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
train_dataset = datasets.ImageFolder(root='./lenses/train', #3001 images each
                                           transform=data_transform)
test_dataset = datasets.ImageFolder(root='./lenses/test', #2000 images each
                                           transform=data_transform)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)

test_loader = torch.utils.data.DataLoader(
        test_dataset, BATCH_SIZE, shuffle=False, num_workers=2
)

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=MOMENTUM)

In [ ]:
# model = make_model(
#     MODEL_NAME, 
#     num_classes=len(CLASSES), 
#     pretrained=True, 
#     input_size=(150, 150)
# ) 
# need to downgrade pytorch ver below 0.4 in order to use cnn_finetune

model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, len(CLASSES))

# model_ft = models.resnet18(pretrained=use_pretrained)
#         set_parameter_requires_grad(model_ft, feature_extract)
#         num_ftrs = model_ft.fc.in_features
#         model_ft.fc = nn.Linear(num_ftrs, num_classes)
#         input_size = 224

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
#Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

In [ ]:
model_ft = model.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=N_EPOCHS, is_inception=False)

In [ ]:
# for cnn_finetune
# Use exponential decay for fine-tuning optimizer
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.975)

# Train
for epoch in range(1, N_EPOCHS + 1):
    # Decay Learning Rate
    scheduler.step(epoch)
    train(model, epoch, optimizer, train_loader)
    test(model, test_loader)
